In [1]:
import cv2
import numpy as np

In [3]:
win_name = 'scan'
# Read image
img = cv2.imread('./img/paper.jpg')
cv2.imshow('original', img)
cv2.waitKey()
draw = img.copy()

# Convert to gray scale and canny edge
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)    # Remove noise using gaussian blur
edged = cv2.Canny(gray, 75, 200)            # Edge detection with Canny edge
cv2.imshow(win_name, edged)
cv2.waitKey()

# Find contour
contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]

# Draw all contours
cv2.drawContours(draw, contours, -1, (0, 255, 0))
cv2.imshow(win_name, draw)
cv2.waitKey()

# Sort contours by size of area
conts = sorted(contours, key = cv2.contourArea, reverse=True)[:5]
for c in conts:
    # Approximate contour by simplifying from the contour with the largest area
    peri = cv2.arcLength(c, True)    # Perimeter length
    # Approximate to the nearest 0.02 of the perimeter length
    vertices = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(vertices) == 4:           # Break if there are 4 vertices
        break
pts = vertices.reshape(4, 2)    # N x 1 x 2 -> 4 x 2 resize
for x, y in pts:
    cv2.circle(draw, (x, y), 10, (0, 255, 0), -1)    # Draw green circle at the coordinates
cv2.imshow(win_name, draw)
cv2.waitKey()
merged = np.hstack((img, draw))

### Same as Example 5-18 below: ###
# Find the top, bottom, left, and right coordinates among four coordinates
sm = pts.sum(axis=1)              # 4쌍의 좌표 각각 x+y 계산
diff = np.diff(pts, axis = 1)     # 4쌍의 좌표 각각 x-y 계산

topLeft = pts[np.argmin(sm)]      # x+y가 가장 작은 값이 좌상단 좌표
bottomRight = pts[np.argmax(sm)]  # x+y가 가장 큰 값이 우하단 좌표
topRight = pts[np.argmin(diff)]   # x-y가 가장 작은 값이 우상단 좌표
bottomLeft = pts[np.argmax(diff)] # x-y가 가장 큰 값이 좌하단 좌표

# four coordinates before conversion
pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])

# Calculate the width and height of the document to be used in the image after conversion
w1 = abs(bottomRight[0] - bottomLeft[0])    # 하단 좌우 좌표 거리
w2 = abs(topRight[0] - topLeft[0])          # 상단 좌우 좌표 거리
h1 = abs(topRight[1] - bottomRight[1])      # 우측 상하 좌표 거리
h2 = abs(topLeft[1] - bottomLeft[1])        # 좌측 상하 좌표 거리
width = max([w1, w2])                       # 두 좌우 거리 간의 최대 값이 서류의 폭
height = max([h1, h2])                      # 두 상하 거리 간의 최대 값이 서류의 높이

# four coordinates after conversion
pts2 = np.float32([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]])

# Calculation of transformation matrix
mtrx = cv2.getPerspectiveTransform(pts1, pts2)

# Apply perspective transformation
result = cv2.warpPerspective(img, mtrx, (int(width), int(height)))

cv2.imshow(win_name, result)
cv2.waitKey()
cv2.destroyAllWindows()